In [2]:
import torch
import soundfile as sf
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.api import TTS

from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

Flash attention 2 is not installed


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
version = "2" # Used when specifying output path of audio files, increment as needed

In [4]:
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text="At a certain point you can't keep blaming your childhood for how you act now. Yes, your parents messed up. Yes, you grew up in a tough environment. Yes, you learned unhealthy patterns early. All of that shaped you. But it doesn't get to control you forever. There comes a time when how you treat people is on you. When you're reactions are your choice. When continuing the cycle becomes your decision, not your pasts. You could understand why you're angry, without taking it out on others. You could know why you struggle with trust without punishing everyone who tries to get close. You can see where your issues come from, without using them as an excuse. Your trauma explains your behavior, it doesn't excuse it. At some point you have to stop being a victim of you past and start taking responsibility for your present. What happened to you, wasn't your fault. What you do now is.",
                file_path="./audios/ttsAPI" + version + ".wav",
                speaker_wav="./voices/longvoice1.wav",
                language="en")

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


c:\Users\jcan2\GitHub\ReelGenerator\.venv\Lib\site-packages\TTS\tts\layers\xtts\xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(s

 > Text splitted to sentences.
["At a certain point you can't keep blaming your childhood for how you act now.", 'Yes, your parents messed up.', 'Yes, you grew up in a tough environment.', 'Yes, you learned unhealthy patterns early.', 'All of that shaped you.', "But it doesn't get to control you forever.", 'There comes a time when how you treat people is on you.', "When you're reactions are your choice.", 'When continuing the cycle becomes your decision, not your pasts.', "You could understand why you're angry, without taking it out on others.", 'You could know why you struggle with trust without punishing everyone who tries to get close.', 'You can see where your issues come from, without using them as an excuse.', "Your trauma explains your behavior, it doesn't excuse it.", 'At some point you have to stop being a victim of you past and start taking responsibility for your present.', "What happened to you, wasn't your fault.", 'What you do now is.']


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 > Processing time: 301.78982400894165
 > Real-time factor: 4.184482666736988


'./audios/ttsAPI2.wav'

In [14]:
MODEL_DIR = "./models/XTTS-v2"  # folder containing config.json, model.pth, speakers_xtts.pth, vocab.json, etc.

config = XttsConfig()
config.load_json(f"{MODEL_DIR}/config.json")

xttsModel = Xtts.init_from_config(config)
xttsModel.load_checkpoint(config, checkpoint_dir=MODEL_DIR, eval=True)

xttsModel = xttsModel.to(device)

outputs = xttsModel.synthesize(
    "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
    config,
    speaker_wav="./voices/longvoice1.wav",
    language="en",
)

sf.write("./audios/ttsSaved" + version + ".wav", outputs["wav"], config.audio.sample_rate)

In [ ]:
parlerModel = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

prompt = """
I've made so many mistakes.
I'm completely lost in life.
Everything's falling apart.
Stop thinking negatively.
The past is in the past.
The future is in the future.
I just need to focus on today.
"""
description = "A burly man's voice is monotone yet warm, deep, inspiring, with a very close recording that almost has no background noise."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = parlerModel.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("./audios/parler_tts_out.wav", audio_arr, parlerModel.config.sampling_rate)